# Task: Spelling Correction

Given a word *w*, find the most likely correction *c* = `correct(`*w*`)`.

**Approach:** Try all candidate words *c* that are known words that are near *w*.  Choose the most likely one.

How to balance *near* and *likely*?

For now, in a trivial way: always prefer nearer, but when there is a tie on nearness, use the word with the highest `WORDS` count.  Measure nearness by *edit distance*: the minimum number of deletions, transpositions, insertions, or replacements of characters. By trial and error, we determine that going out to edit distance 2 will give us reasonable results.  Then we can define `correct(`*w*`)`:

In [ ]:
import nltk

import re
import collections
from google.colab import files

In [ ]:
def tokens(text):
    """
    Get all words from the corpus
    """
    return re.findall('[a-z]+', text.lower())

In [ ]:
TEXT = open('big.txt').read()
len(TEXT)

WORDS = tokens(TEXT)
WORD_COUNTS = collections.Counter(WORDS)
print(WORD_COUNTS.most_common(10))

[('the', 80030), ('of', 40025), ('and', 38313), ('to', 28766), ('in', 22050), ('a', 21155), ('that', 12512), ('he', 12401), ('was', 11410), ('it', 10681)]


Now for `edits1(word)`: the set of candidate words that are one edit away. For example, given `"wird"`, this would include `"weird"` (inserting an `e`) and `"word"` (replacing a `i` with a `o`), and also `"iwrd"` (transposing `w` and `i`; then `known` can be used to filter this out of the set of final candidates). How could we get them?  One way is to *split* the original word in all possible places, each split forming a *pair* of words, `(a, b)`, before and after the place, and at each place, either delete, transpose, replace, or insert a letter:

<table>
  <tr><td> pairs: <td><tt> Ø, wird <td><tt> w, ird <td><tt> wi, rd <td><tt>wir, d<td><tt>wird, Ø<td><i>Notes:</i><tt> (<i>a</i>, <i>b</i>)</tt> pair</i>
  <tr><td> deletions: <td><tt>Ø+ird<td><tt> w+rd<td><tt> wi+d<td><tt> wir+Ø<td><td><i>Delete first char of b</i>
  <tr><td> transpositions: <td><tt>Ø+iwrd<td><tt> w+rid<td><tt> wi+dr</tt><td><td><td><i>Swap first two chars of b
  <tr><td> replacements: <td><tt>Ø+?ird<td><tt> w+?rd<td><tt> wi+?d<td><tt> wir+?</tt><td><td><i>Replace char at start of b
  <tr><td> insertions: <td><tt>Ø+?+wird<td><tt> w+?+ird<td><tt> wi+?+rd<td><tt> wir+?+d<td><tt> wird+?+Ø</tt><td><i>Insert char between a and b
</table>

In [ ]:


def edits0(word):
    """
    Return all strings that are zero edits away
    from the input word (i.e., the word itself).
    """
    return {word}

def edits1(word):
    """
    Return all strings that are one edit away
    from the input word.
    """
    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    def splits(word):
        """
        Return a list of all possible (first, rest) pairs
        that the input word is made of.
        """
        return [(word[:i], word[i:])
                for i in range(len(word)+1)]

    pairs = splits(word)
    deletes = [a + b[1:] for (a, b) in pairs if b]
    transposes = [a + b[1] + b[0] + b[2:] for (a, b) in pairs if len(b) > 1]
    replaces = [a + c + b[1:] for (a, b) in pairs for c in alphabet
                if b]
    inserts = [a + c + b for (a, b) in pairs for c in alphabet]
    return set(deletes + transposes + replaces + inserts)


def edits2(word):
    """Return all strings that are two edits away
    from the input word.
    """
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)}

def known(words):
    """
    Return the subset of words that are actually
    in our WORD_COUNTS dictionary.
    """
    return {w for w in words if w in WORD_COUNTS}

In [ ]:
word = 'fianlly'
# zero edit distance from input word
print(edits0(word))
# returns null set since it is not a valid word
print(known(edits0(word)))
# one edit distance from input word
print(edits1(word))
# get correct words from above set
print(known(edits1(word)))
# two edit distances from input word
print(edits2(word))
# get correct words from above set
print(known(edits2(word)))


{'fianlly'}
set()
{'fianllyw', 'fnianlly', 'fianelly', 'tfianlly', 'fianlhy', 'fkianlly', 'fgianlly', 'nfianlly', 'fiaylly', 'fieanlly', 'fianllly', 'fiajnlly', 'sfianlly', 'fianllxy', 'fianply', 'cfianlly', 'fivanlly', 'fimanlly', 'fiaally', 'fianllq', 'faanlly', 'fifnlly', 'fiajlly', 'fiaclly', 'fianlsly', 'ifianlly', 'fiabnlly', 'fianllyx', 'feanlly', 'fianxly', 'fianllmy', 'xianlly', 'fpianlly', 'fianaly', 'fianlely', 'fiagnlly', 'fianllfy', 'iianlly', 'firanlly', 'fqianlly', 'fijanlly', 'fbianlly', 'fianlvy', 'fianllwy', 'fianlloy', 'finnlly', 'fibanlly', 'fianllo', 'fiaully', 'zfianlly', 'fianjly', 'ianlly', 'fianqlly', 'fianlyl', 'fianlgly', 'fianllyy', 'fiansly', 'fianltly', 'fanlly', 'fsanlly', 'fiinlly', 'fzianlly', 'fbanlly', 'fianlll', 'fianllyb', 'fianlcy', 'jfianlly', 'fianllyk', 'fiamnlly', 'fzanlly', 'fiaonlly', 'fianllyn', 'fianllky', 'qfianlly', 'fianzly', 'fianllh', 'ufianlly', 'pianlly', 'fianlmly', 'fianlli', 'fianluly', 'fianlxy', 'fianclly', 'fianlbly', 'fidanlly

In [ ]:
candidates = (known(edits0(word)) or known(edits1(word)) or known(edits2(word)) or [word])

print(candidates)

{'finally'}


In [ ]:
def correct(word):
    """
    Get the best correct spelling for the input word
    """
    # Priority is for edit distance 0, then 1, then 2
    # else defaults to the input word itself.
    candidates = (known(edits0(word)) or
                  known(edits1(word)) or
                  known(edits2(word)) or
                  [word])
    return max(candidates, key=WORD_COUNTS.get)

print(correct('fianlly'))
print(correct('FIANLLY'))



finally
FIANLLY


In [ ]:
def correct_match(match):
    """
    Spell-correct word in match,
    and preserve proper upper/lower/title case.
    """
    word = match.group()
    def case_of(text):
        """
        Return the case-function appropriate
        for text: upper, lower, title, or just str.:
            """
        return (str.upper if text.isupper() else
                str.lower if text.islower() else
                str.title if text.istitle() else
                str)
    return case_of(word)(correct(word.lower()))

def correct_text_generic(text):
    """
    Correct all the words within a text,
    returning the corrected text.
    """
    return re.sub('[a-zA-Z]+', correct_match, text)

print(correct_text_generic('fianlly'))

print(correct_text_generic('FIANLLY'))

finally
FINALLY


In [ ]:
known(edits1('wird'))

{'bird',
 'gird',
 'sird',
 'ward',
 'weird',
 'wid',
 'wild',
 'wind',
 'wire',
 'wired',
 'wirt',
 'wiry',
 'word'}

In [ ]:
s = 'Speling ERRURS in "somethink." Whutever; unusuel misteakes everyware?'

{w: correct(w) for w in tokens(s)}

{'errurs': 'errors',
 'everyware': 'everywhere',
 'in': 'in',
 'misteakes': 'mistakes',
 'somethink': 'something',
 'speling': 'spelling',
 'unusuel': 'unusual',
 'whutever': 'whatever'}